In [81]:
import pandas as pd
import scipy.stats as st
import numpy as np
path_output_teds_dndscv="/home/fran/Documents/cuppen/HPC/tunnel//cuppen/projects/P0025_PCAWG_HMF/drivers/processed/resistance/dndscv/"

### Analysis for the Supp. Note 4 (TEDs)

In [2]:
results = pd.read_csv("../results/data/table_all_TEDS.tsv",sep="\t")

In [7]:
results[results["is_target_treatment"]][["gene","drug","cancer_type_code"]].drop_duplicates()

,gene,drug,cancer_type_code
4,ERBB2,Anti_HER2,BRCA
8,EGFR,Anti_EGFR,LUAD
25,BRAF,BRAF_inhibitor,SKCM


In [ ]:
def count_unique(grp):
    return len(set(grp))

##### For those cases that the TED is also a target of primary treatment (eg, EGFR mutated),  identify secondary mutations that are recurrently enriched in the treatment group

In [97]:
l=[]
for _,r in results[results["is_target_treatment"]][["gene","drug","cancer_type_code"]].drop_duplicates().iterrows():
    # gather the untreated group 
    treated_samples = f"{path_output_teds_dndscv}/{r['cancer_type_code']}/{r['drug']}.dndscv.annotmuts.tsv.gz"
    untreated_samples = f"{path_output_teds_dndscv}/{r['cancer_type_code']}/untreated.dndscv.annotmuts.tsv.gz"
    df_un = pd.read_csv(untreated_samples,sep="\t")
    df_un = df_un[(df_un["gene"]==r["gene"])&(df_un["impact"]!="Synonymous")]
    samples_untreated=df_un.shape[0]
    df_tr = pd.read_csv(treated_samples,sep="\t")
    df_tr = df_tr[(df_tr["gene"]==r["gene"])&(df_tr["impact"]!="Synonymous")]
    samples_treated=df_tr.shape[0]
    x=df_tr.groupby(["aachange"],as_index=False).agg({"sampleID":count_unique})
    y=df_un.groupby(["aachange"],as_index=False).agg({"sampleID":count_unique})
    rec_treated_muts=x[x["sampleID"]>1]
    for _,rm in rec_treated_muts.iterrows():
        sun=y[y["aachange"]==rm["aachange"]]
        samples_untreated_mut=0 # this is allowed when reliying on G-test
        if sun.shape[0]>0:
            samples_untreated_mut=int(sun["sampleID"])
        samples_treated_mut = rm["sampleID"]
        stat,pvalue,dof,expected=st.chi2_contingency(np.array([[samples_treated_mut,samples_treated],[samples_untreated_mut,samples_untreated]]),lambda_ = 'log-likelihood')
        odds_ratio = (samples_treated_mut/samples_treated) / (np.nanmax([samples_untreated_mut,0.1])/samples_untreated)
        l.append([r["gene"],rm["aachange"],r["drug"],r["cancer_type_code"]]+[samples_treated_mut,samples_treated]+[samples_untreated_mut,samples_untreated]+[odds_ratio,stat,pvalue])           
df_results = pd.DataFrame(l,columns=["gene","mutation","drug","cancer_type_code","samples_treated_mutation_found","total_samples_treated_gene_mutated","samples_untreated_mutation_found","total_samples_untreated_gene_mutated","odds_ratio","G","G-test pvalue"])  
df_results["is_exlusive_treatment"] = df_results.apply(lambda row: row["samples_untreated_mutation_found"] ==0,axis=1)

In [99]:
df_results.sort

,gene,mutation,drug,cancer_type_code,samples_treated_mutation_found,total_samples_treated_gene_mutated,samples_untreated_mutation_found,total_samples_untreated_gene_mutated,odds_ratio,G,G-test pvalue,is_exlusive_treatment
0,ERBB2,L755S,Anti_HER2,BRCA,2,12,0,3,5.000000,0.077798,0.780304,True
1,ERBB2,V777L,Anti_HER2,BRCA,2,12,0,3,5.000000,0.077798,0.780304,True
2,EGFR,.,Anti_EGFR,LUAD,20,78,3,6,0.512821,0.217129,0.641237,False
3,EGFR,A750P,Anti_EGFR,LUAD,2,78,0,6,1.538462,0.654510,0.418505,True
4,EGFR,C797S,Anti_EGFR,LUAD,8,78,0,6,6.153846,0.001074,0.973857,True
5,EGFR,G719A,Anti_EGFR,LUAD,2,78,0,6,1.538462,0.654510,0.418505,True
6,EGFR,G719S,Anti_EGFR,LUAD,2,78,0,6,1.538462,0.654510,0.418505,True
7,EGFR,L858R,Anti_EGFR,LUAD,12,78,0,6,9.230769,0.112564,0.737243,True
8,EGFR,L861Q,Anti_EGFR,LUAD,2,78,1,6,0.153846,0.246928,0.619247,False
9,EGFR,T790M,Anti_EGFR,LUAD,18,78,0,6,13.846154,0.450479,0.502108,True


In [98]:
df_results.to_csv("../results/table_muts_enrichment_TEDs_targets.tsv",sep="\t",index=False)